In [1]:
%pwd

'c:\\Users\\Kashan\\Desktop\\Medical_Chatbot\\Gen_Ai_Medical_Chatbot\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\Kashan\\Desktop\\Medical_Chatbot\\Gen_Ai_Medical_Chatbot'

In [4]:
%cd Gen_Ai_Medical_Chatbot/

[WinError 2] The system cannot find the file specified: 'Gen_Ai_Medical_Chatbot/'
c:\Users\Kashan\Desktop\Medical_Chatbot\Gen_Ai_Medical_Chatbot\research


In [5]:
%pwd

'c:\\Users\\Kashan\\Desktop\\Medical_Chatbot\\Gen_Ai_Medical_Chatbot'

In [6]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [7]:
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob = "*.pdf",
                             loader_cls = PyPDFLoader)
    documents = loader.load()
    
    return documents 

In [8]:
extracted_data = load_pdf_file(data = "Data/")

In [9]:
#extracted_data

In [10]:
# making chunks of the extracted data
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks = text_split(extracted_data)
print(f"length of text_chunks is : {len(text_chunks)}")


length of text_chunks is : 5860


In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

In [13]:
#download an embedding model from HuggingFace

def get_embedding_model():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [14]:
embeddings = get_embedding_model()

C:\Users\Kashan\AppData\Local\Temp\ipykernel_2452\3033671372.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\Kashan\miniconda3\envs\medbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
query_result = embeddings.embed_query('Hello World')
print(f"Length: {len(query_result)}")

Length: 384


In [16]:
#query_result

In [31]:
from dotenv import load_dotenv
load_dotenv()


True

In [32]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [33]:
from pinecone.grpc import PineconeGRPC as pinecone
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)



index_name = "medbot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '547fba354e56a325d1e7e0aebf5d841d', 'Date': 'Thu, 14 Nov 2024 10:30:49 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [34]:
import os
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [21]:
#Embed each chunk and insert it into vector database

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks,
    embedding = embeddings,
    index_name = index_name
)

KeyboardInterrupt: 

In [35]:
##Load Existing Index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    embedding = embeddings,
    index_name = index_name
)

In [36]:
docsearch

In [41]:
retriever = docsearch.as_retriever(search_type = "similarity", search_kwags = {"k":3})

In [42]:
retrieved_docs = retriever.invoke("what is hernia?")

In [43]:
retrieved_docs

[Document(id='afff9d0d-19c3-446d-9a56-64b410a7250e', metadata={'page': 19.0, 'source': 'Data\\Medical_Book.pdf'}, page_content='The condition is, in fact, a hernia requiring only replace-\nment and strengthening of the passageway through\nwhich it occurred. After surgery, increased pressure in\nthe stretched abdomen can compromise the function of\nthe organs inside.\nPrevention\nSome, but by no means all, birth defects are pre-\nventable by early and attentive prenatal care, good nutri-\ntion, supplemental vitamins , diligent avoidance of all\nunnecessary drugs and chemicals—especially tobacco—'),
 Document(id='73ccc13a-6a3f-4cde-ad33-27baf6c0d566', metadata={'page': 19.0, 'source': 'Data\\Medical_Book.pdf'}, page_content='The condition is, in fact, a hernia requiring only replace-\nment and strengthening of the passageway through\nwhich it occurred. After surgery, increased pressure in\nthe stretched abdomen can compromise the function of\nthe organs inside.\nPrevention\nSome, but by 

In [44]:
from langchain_openai import OpenAI
llm = OpenAI(temperature = 0.4, max_tokens = 500)

In [45]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt= (
    "You are a conversational AI. I am a user. I am asking you a question. "
    "use the retrieved information to answer my question. "
    "if you dont know the answer, say that you dont know. "
    "Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"   
)


prompt = ChatPromptTemplate.from_messages(
    
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [46]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain) 

In [48]:
response = rag_chain.invoke({"input": "what is hernia?"})
print(response["answer"])



A hernia is a condition where a structure, such as an organ, moves into a place where it does not belong. It often requires surgery to replace and strengthen the passageway where it occurred. In some cases, increased pressure in the abdomen can affect the function of the organs inside.


In [49]:
response = rag_chain.invoke({"input": "what is stats?"})
print(response["answer"])



I am not familiar with the term "stats." Can you provide more context or information for me to better understand and answer your question?


In [51]:
response = rag_chain.invoke({"input": "what is gigantism?"})
print(response["answer"])



Gigantism is a disorder in which the body experiences abnormal growth, resulting in an individual being much taller and larger than average. It is caused by an overproduction of growth hormone from the pituitary gland in the brain. Symptoms may include excessive height, enlarged hands and feet, and other physical abnormalities. Treatment options include medications and surgery to remove the pituitary tumor causing the excess growth hormone.
